In [2]:
import pandas as pd

df = pd.read_csv("itineraries.csv", nrows=5)
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0,False,False,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0,False,False,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0,False,False,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach


In [3]:
from collections import Counter

route_counts = Counter()

for chunk in pd.read_csv("itineraries.csv", usecols=["startingAirport", "destinationAirport"], chunksize=500_000):
    chunk = chunk.dropna(subset=["startingAirport", "destinationAirport"])
    
    routes = chunk["startingAirport"] + "-" + chunk["destinationAirport"]
    
    route_counts.update(routes)

route_df = pd.DataFrame(route_counts.most_common(20), columns=["Route", "Count"])
print(route_df)


KeyboardInterrupt: 

In [ ]:
origin = "LAX"
destination = "JFK"

airline_counts = Counter()

for chunk in pd.read_csv("itineraries.csv", usecols=["startingAirport", "destinationAirport", "segmentsAirlineName"], chunksize=500_000):
    chunk = chunk.dropna(subset=["startingAirport", "destinationAirport", "segmentsAirlineName"])
    
    mask = (chunk["startingAirport"] == origin) & (chunk["destinationAirport"] == destination)
    airline_counts.update(chunk.loc[mask, "segmentsAirlineName"])

airline_df = pd.DataFrame(airline_counts.most_common(), columns=["Airline", "FlightCount"])
print(airline_df.head(10))


                                Airline  FlightCount
0  American Airlines||American Airlines       151007
1                       JetBlue Airways       113339
2                     American Airlines        93532
3      Alaska Airlines||Alaska Airlines        84522
4                                 Delta        71229
5                          Delta||Delta        41616
6      JetBlue Airways||JetBlue Airways        23101
7                                United        16705
8                        United||United        14394
9               United||Alaska Airlines         7268


In [4]:
import pandas as pd

# Clean airline names
airline_df["Airline"] = airline_df["Airline"].apply(lambda x: x.split("||")[0].strip())

# Re-sum counts for duplicate names
airline_df = airline_df.groupby("Airline", as_index=False)["FlightCount"].sum()

# Sort by count again
airline_df = airline_df.sort_values(by="FlightCount", ascending=False).reset_index(drop=True)

print(airline_df.head(10))


NameError: name 'airline_df' is not defined

In [5]:
origin = "LAX"
destination = "JFK"
airline = "American Airlines"

chunks = []

for chunk in pd.read_csv("itineraries.csv", chunksize=500_000):
    mask = (
        (chunk["startingAirport"] == origin)
        & (chunk["destinationAirport"] == destination)
        & (chunk["segmentsAirlineName"].str.contains(airline, case=False, na=False))
    )
    filtered = chunk.loc[mask].copy()
    if not filtered.empty:
        chunks.append(filtered)

df = pd.concat(chunks)
df.to_csv("aa_lax_jfk_full.csv", index=False)
print("✅ Saved full filtered dataset with shape:", df.shape)

KeyboardInterrupt: 

In [6]:

df = pd.read_csv("aa_lax_jfk_full.csv")
print("Dataset shape:", df.shape)

Dataset shape: (246932, 27)


In [7]:
missing = df.isnull().sum().sort_values(ascending=False)
missing_percent = (missing / len(df)) * 100

missing_summary = pd.DataFrame({
    "Missing Values": missing,
    "Percent Missing": missing_percent.round(2)
})

print(missing_summary.head(15))

                     Missing Values  Percent Missing
totalTravelDistance            1432             0.58
segmentsDistance                358             0.14
legId                             0             0.00
startingAirport                   0             0.00
destinationAirport                0             0.00
fareBasisCode                     0             0.00
travelDuration                    0             0.00
elapsedDays                       0             0.00
isBasicEconomy                    0             0.00
searchDate                        0             0.00
flightDate                        0             0.00
isNonStop                         0             0.00
isRefundable                      0             0.00
totalFare                         0             0.00
baseFare                          0             0.00


In [8]:
df = df.dropna(subset=["totalTravelDistance", "segmentsDistance"])

In [9]:
df = df.drop_duplicates()

df = df[df["flightDate"] >= df["searchDate"]]

df.to_csv("aa_lax_jfk_clean.csv", index=False)
print("✅ Cleaned dataset saved:", df.shape)

✅ Cleaned dataset saved: (245500, 27)


In [10]:
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,ef15f068409b3ac81252397357f64341,2022-04-16,2022-04-17,LAX,JFK,M0AHZNN1,PT5H29M,1,False,False,...,1650256140,2022-04-18T00:29:00.000-04:00,JFK,LAX,American Airlines,AA,AIRBUS INDUSTRIE A321 SHARKLETS,19740,2458,coach
1,37ba57ecb1ff9dda44cc8ecb7fa4c78f,2022-04-16,2022-04-17,LAX,JFK,M0AHZNN1,PT5H29M,1,False,False,...,1650284880,2022-04-18T08:28:00.000-04:00,JFK,LAX,American Airlines,AA,AIRBUS INDUSTRIE A321 SHARKLETS,19740,2458,coach
2,f6f6607cd706c732459d2853ab445aee,2022-04-16,2022-04-17,LAX,JFK,M0AHZNN1,PT5H30M,0,False,False,...,1650238200,2022-04-17T19:30:00.000-04:00,JFK,LAX,American Airlines,AA,AIRBUS INDUSTRIE A321 SHARKLETS,19800,2458,coach
3,c284f0239f46fd2afd6d72a33c2737f1,2022-04-16,2022-04-17,LAX,JFK,M0AHZNN1,PT5H30M,0,False,False,...,1650241800,2022-04-17T20:30:00.000-04:00,JFK,LAX,American Airlines,AA,AIRBUS INDUSTRIE A321 SHARKLETS,19800,2458,coach
4,7558638d8c2c18d062c73bc7abb06a00,2022-04-16,2022-04-17,LAX,JFK,M0AHZNN1,PT5H30M,0,False,False,...,1650225600,2022-04-17T16:00:00.000-04:00,JFK,LAX,American Airlines,AA,AIRBUS INDUSTRIE A321 SHARKLETS,19800,2458,coach


In [11]:
cols = pd.read_csv("itineraries.csv", nrows=0).columns.tolist()
print("Number of columns:", len(cols))
print(cols)

Number of columns: 27
['legId', 'searchDate', 'flightDate', 'startingAirport', 'destinationAirport', 'fareBasisCode', 'travelDuration', 'elapsedDays', 'isBasicEconomy', 'isRefundable', 'isNonStop', 'baseFare', 'totalFare', 'seatsRemaining', 'totalTravelDistance', 'segmentsDepartureTimeEpochSeconds', 'segmentsDepartureTimeRaw', 'segmentsArrivalTimeEpochSeconds', 'segmentsArrivalTimeRaw', 'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode', 'segmentsAirlineName', 'segmentsAirlineCode', 'segmentsEquipmentDescription', 'segmentsDurationInSeconds', 'segmentsDistance', 'segmentsCabinCode']


In [12]:
df[["baseFare", "totalFare"]].head()


,baseFare,totalFare
0,327.44,366.6
1,327.44,366.6
2,327.44,366.6
3,327.44,366.6
4,327.44,366.6


In [13]:
df["segmentsCabinCode"].unique()

array(['coach', 'coach||coach', 'coach||coach||coach', 'business',
       'first', 'coach||first||coach', 'business||coach',
       'first||coach||first', 'coach||first'], dtype=object)

In [14]:
df["isNonStop"].value_counts()

isNonStop
False    152326
True      93174
Name: count, dtype: int64

In [18]:
df.columns.tolist()


['legId',
 'searchDate',
 'flightDate',
 'startingAirport',
 'destinationAirport',
 'fareBasisCode',
 'travelDuration',
 'elapsedDays',
 'isBasicEconomy',
 'isRefundable',
 'isNonStop',
 'baseFare',
 'totalFare',
 'seatsRemaining',
 'totalTravelDistance',
 'segmentsDepartureTimeEpochSeconds',
 'segmentsDepartureTimeRaw',
 'segmentsArrivalTimeEpochSeconds',
 'segmentsArrivalTimeRaw',
 'segmentsArrivalAirportCode',
 'segmentsDepartureAirportCode',
 'segmentsAirlineName',
 'segmentsAirlineCode',
 'segmentsEquipmentDescription',
 'segmentsDurationInSeconds',
 'segmentsDistance',
 'segmentsCabinCode']

In [19]:
def simplify_cabin(cabin):
    cabin = cabin.lower()
    if "first" in cabin:
        return "first"
    elif "business" in cabin:
        return "business"
    else:
        return "coach"

df["mainCabin"] = df["segmentsCabinCode"].apply(simplify_cabin)

In [20]:
df["mainCabin"].head()

0    coach
1    coach
2    coach
3    coach
4    coach
Name: mainCabin, dtype: object

In [ ]:
df["mainCabin"].value_counts()

mainCabin
coach       244584
business       567
first          349
Name: count, dtype: int64

In [22]:
avg_fares = df.groupby("mainCabin")["totalFare"].mean().sort_values()
print(avg_fares)

mainCabin
coach        453.606762
business    2641.191182
first       4044.816762
Name: totalFare, dtype: float64


In [28]:
avg_fare_by_cabin = df.groupby("mainCabin")["totalFare"].mean()
coach_price = avg_fare_by_cabin["coach"]

price_ratios = (avg_fare_by_cabin / coach_price).round(2)
print(price_ratios)


mainCabin
business    5.82
coach       1.00
first       8.92
Name: totalFare, dtype: float64


In [29]:
selected_columns = ["searchDate", "flightDate", "totalFare", "mainCabin", "isNonStop"]
df_subset = df[selected_columns].copy()

In [30]:
df_subset.head()
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245500 entries, 0 to 246931
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   searchDate  245500 non-null  object 
 1   flightDate  245500 non-null  object 
 2   totalFare   245500 non-null  float64
 3   mainCabin   245500 non-null  object 
 4   isNonStop   245500 non-null  bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 9.6+ MB


In [31]:
df_subset["searchDate"] = pd.to_datetime(df_subset["searchDate"])
df_subset["flightDate"] = pd.to_datetime(df_subset["flightDate"])

In [32]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245500 entries, 0 to 246931
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   searchDate  245500 non-null  datetime64[ns]
 1   flightDate  245500 non-null  datetime64[ns]
 2   totalFare   245500 non-null  float64       
 3   mainCabin   245500 non-null  object        
 4   isNonStop   245500 non-null  bool          
dtypes: bool(1), datetime64[ns](2), float64(1), object(1)
memory usage: 9.6+ MB


In [33]:
df_subset.to_csv("aa_lax_jfk_filtered.csv", index=False)
print("✅ Clean filtered dataset saved successfully!")

✅ Clean filtered dataset saved successfully!
